In [ ]:
from torch.utils.data import DataLoader
from torchvision import datasets, transforms


def get_mnist_dataloaders(batch_size=128):
    """MNIST dataloader with (32, 32) sized images."""
    # Resize images so they are a power of 2
    all_transforms = transforms.Compose([
        transforms.Resize(32),
        transforms.ToTensor()
    ])
    # Get train and test data
    train_data = datasets.MNIST('../data', train=True, download=True,
                                transform=all_transforms)
    test_data = datasets.MNIST('../data', train=False,
                               transform=all_transforms)
    # Create dataloaders
    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=True)
    return train_loader, test_loader


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Generator(nn.Module):

    def __init__(self, img_size, latent_dim, dim):
        super(Generator, self).__init__()

        self.dim = dim
        self.latent_dim = latent_dim
        self.img_size = img_size
        self.feature_sizes = (int(self.img_size[0] / 16), int(self.img_size[1] / 16))
        #print(self.feature_sizes)

        self.latent_to_features = nn.Sequential(
            nn.Linear(latent_dim, 8 * dim * self.feature_sizes[0] * self.feature_sizes[1]),
            nn.ReLU(inplace=True)
        )

        self.features_to_image = nn.Sequential(
            nn.ConvTranspose2d(8 * dim, 4 * dim, 4, 2, 1),
            nn.ReLU(),
            nn.BatchNorm2d(4 * dim),
            nn.ConvTranspose2d(4 * dim, 2 * dim, 4, 2, 1),
            nn.ReLU(),
            nn.BatchNorm2d(2 * dim),
            nn.ConvTranspose2d(2 * dim, dim, 4, 2, 1),
            nn.ReLU(),
            nn.BatchNorm2d(dim),
            nn.ConvTranspose2d(dim, self.img_size[2], 4, 2, 1),
            nn.Sigmoid()
        )

    def forward(self, x):

        x = self.latent_to_features(x)
        x = x.view(-1, 8 * self.dim, self.feature_sizes[0], self.feature_sizes[1])

        return self.features_to_image(x)

    def sample_latent(self, num_samples):

        return torch.randn((num_samples, self.latent_dim))



class Discriminator(nn.Module):

    def __init__(self, img_size, dim):
        super(Discriminator, self).__init__()

        self.img_size = img_size

        self.image_to_features = nn.Sequential(
            nn.Conv2d(self.img_size[2], dim, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(dim, dim * 2, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(dim * 2, dim * 4, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(dim * 4, dim * 8, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Sigmoid()
        )

        output_size = 8 * dim * int(img_size[0] / 16) * int(img_size[1] / 16)

        self.features_to_prob = nn.Sequential(
            nn.Linear(output_size, 1),
            nn.Sigmoid()
        )

    def forward(self, x):

        batch_size = x.size()[0]
        x = self.image_to_features(x)
        x = x.view(batch_size, -1)

        return self.features_to_prob(x)

In [ ]:
import imageio
import numpy as np
import torch
import torch.nn as nn
from torchvision.utils import make_grid
from torch.autograd import grad
import torch.optim as optim
import torch.autograd as autograd

class Trainer():

    def __init__(self, generator, discriminator, gen_optimizer, dis_optimizer,
                 gp_weight=10, critic_iteration=5, print_every=50, use_cuda=True):
        
        self.G = generator
        self.G_optim = gen_optimizer
        self.D = discriminator
        self.D_optim = dis_optimizer
        self.losses = {'G': [], 'D': [], 'GP': [], 'GN': []}
        self.num_steps = 0
        self.use_cuda = use_cuda
        self.gp_weight = gp_weight
        self.critic_iterations = critic_iteration
        self.print_every = print_every

        if self.use_cuda:

            self.G.cuda()
            self.D.cuda()

    def critic_train_iteration(self, data):

        batch_size = data.size()[0]
        generated_data = self.sample_generator(batch_size)

        data = data.cuda()

        d_real = self.D(data)
        d_generated = self.D(generated_data)

        gradient_penalty = self.calc_gp(data, generated_data)
        self.losses['GP'].append(gradient_penalty.item())

        self.D_optim.zero_grad()
        d_loss = d_generated.mean() - d_real.mean() + gradient_penalty
        d_loss.backward()
        self.D_optim.step()

        self.losses['D'].append(d_loss.item())

    def generator_iter(self, data):

        self.G_optim.zero_grad()

        batch_size = data.size()[0]
        generated_data = self.sample_generator(batch_size)

        d_generated = self.D(generated_data)
        g_loss = -d_generated.mean()
        g_loss.backward()
        self.G_optim.step()

        self.losses['G'].append(g_loss.item())

    def calc_gp(self, real_data, generated_data):

        batch_size = real_data.size()[0]
        alpha = torch.randn(batch_size, 1, 1, 1)
        alpha = alpha.expand_as(real_data).cuda()
        interpolated = alpha * real_data.data + (1 - alpha) * generated_data.data
        interpolated = interpolated.cuda()
        interpolated.requires_grad = True

        prob_interpolated = self.D(interpolated)

        gradients = autograd.grad(outputs=prob_interpolated, inputs=interpolated, 
                                  grad_outputs=torch.ones(prob_interpolated.size()).cuda(),
                                  create_graph=True, retain_graph=True)[0]
        
        gradients = gradients.view(batch_size, -1)

        self.losses['GN'].append(gradients.norm(2, dim=1).mean().item())

        grad_norm = torch.sqrt(torch.sum(gradients ** 2, dim=1) + 1e-12)

        return self.gp_weight * ((grad_norm - 1) ** 2).mean()

    def train_epoch(self, data_loader):

        for i, data in enumerate(data_loader):

            self.num_steps += 1
            self.critic_train_iteration(data[0])

            if self.num_steps % self.critic_iterations == 0:
                #print(self.num_steps)
                self.generator_iter(data[0])

            if i % self.print_every == 0 and i != 0:

                print(f"[Step - {i + 1}] [Discriminator Loss - {self.losses['D'][-1]}] [Gradient Penalty - {self.losses['GP'][-1]}] [Generator Loss - {self.losses['G'][-1]}]")

    def train(self, data_loader, epochs, save_training_gif=True):

        if save_training_gif:

            fixed_latents = self.G.sample_latent(64)
            fixed_latents = fixed_latents.cuda()

            training_progress_images = []

        for epoch in range(epochs):

            print(f"\nEpoch {epoch + 1}")
            self.train_epoch(data_loader)

            if save_training_gif:

                img_grid = make_grid(self.G(fixed_latents).cpu().data)
                img_grid = np.transpose(img_grid.numpy(), (1, 2, 0))
                training_progress_images.append(img_grid)

            if save_training_gif:

                imageio.mimsave(f'/content/train_{epoch + 1}_epochs.gif', training_progress_images)

        

        imageio.mimsave(f'/content/train_{epoch + 1}_epochs.gif', training_progress_images)

    def sample_generator(self, num_samples):

        latent_samples = self.G.sample_latent(num_samples)

        if self.use_cuda:
            latent_samples = latent_samples.cuda()

        generated_data = self.G(latent_samples)

        return generated_data



        

        

In [ ]:
data_loader, _ = get_mnist_dataloaders(batch_size=64)
img_size = (32, 32, 1)

generator = Generator(img_size=img_size, latent_dim=100, dim=16)
discriminator = Discriminator(img_size=img_size, dim=16)

lr = 1e-4
betas = (.9, .99)
G_optimizer = optim.Adam(generator.parameters(), lr=lr, betas=betas)
D_optimizer = optim.Adam(discriminator.parameters(), lr=lr, betas=betas)
epochs = 200
trainer = Trainer(generator, discriminator, G_optimizer, D_optimizer)

trainer.train(data_loader, epochs, save_training_gif=True)


Epoch 1
[Step - 51] [Discriminator Loss - 9.743355751037598] [Gradient Penalty - 9.758575439453125] [Generator Loss - -0.49273303151130676
[Step - 101] [Discriminator Loss - 8.078031539916992] [Gradient Penalty - 8.167792320251465] [Generator Loss - -0.48621678352355957
[Step - 151] [Discriminator Loss - 7.035028457641602] [Gradient Penalty - 6.980438232421875] [Generator Loss - -0.5170315504074097
[Step - 201] [Discriminator Loss - 6.224174499511719] [Gradient Penalty - 6.173001289367676] [Generator Loss - -0.502871036529541
[Step - 251] [Discriminator Loss - 5.177618980407715] [Gradient Penalty - 5.059880256652832] [Generator Loss - -0.5287482738494873
[Step - 301] [Discriminator Loss - 5.053575038909912] [Gradient Penalty - 4.878391265869141] [Generator Loss - -0.5203658938407898
[Step - 351] [Discriminator Loss - 4.252845287322998] [Gradient Penalty - 4.180204391479492] [Generator Loss - -0.48926156759262085
[Step - 401] [Discriminator Loss - 3.563399076461792] [Gradient Penalty -


Epoch 2
[Step - 51] [Discriminator Loss - 0.7273311018943787] [Gradient Penalty - 0.8686702847480774] [Generator Loss - -0.444528728723526
[Step - 101] [Discriminator Loss - 1.2902204990386963] [Gradient Penalty - 1.4103753566741943] [Generator Loss - -0.45994359254837036
[Step - 151] [Discriminator Loss - 0.5800164937973022] [Gradient Penalty - 0.8464518189430237] [Generator Loss - -0.43123525381088257
[Step - 201] [Discriminator Loss - 0.21800073981285095] [Gradient Penalty - 0.434285044670105] [Generator Loss - -0.46125730872154236
[Step - 251] [Discriminator Loss - 0.251775324344635] [Gradient Penalty - 0.5120539665222168] [Generator Loss - -0.4184366762638092
[Step - 301] [Discriminator Loss - 0.28002262115478516] [Gradient Penalty - 0.6335687637329102] [Generator Loss - -0.4636070728302002
[Step - 351] [Discriminator Loss - 0.43356117606163025] [Gradient Penalty - 0.709913432598114] [Generator Loss - -0.401254802942276
[Step - 401] [Discriminator Loss - 0.2847508192062378] [Grad


Epoch 3
[Step - 51] [Discriminator Loss - -0.0863642692565918] [Gradient Penalty - 0.2669421434402466] [Generator Loss - -0.37664100527763367
[Step - 101] [Discriminator Loss - 0.018892794847488403] [Gradient Penalty - 0.40237829089164734] [Generator Loss - -0.3550286889076233
[Step - 151] [Discriminator Loss - -0.061950087547302246] [Gradient Penalty - 0.3149038553237915] [Generator Loss - -0.3702506422996521
[Step - 201] [Discriminator Loss - -0.137417733669281] [Gradient Penalty - 0.2838253974914551] [Generator Loss - -0.3441457450389862
[Step - 251] [Discriminator Loss - -0.0483129620552063] [Gradient Penalty - 0.29742127656936646] [Generator Loss - -0.37082821130752563
[Step - 301] [Discriminator Loss - -0.04842463135719299] [Gradient Penalty - 0.32284441590309143] [Generator Loss - -0.36050572991371155
[Step - 351] [Discriminator Loss - -0.08782997727394104] [Gradient Penalty - 0.3100079596042633] [Generator Loss - -0.3540038764476776
[Step - 401] [Discriminator Loss - 0.0258579


Epoch 4
[Step - 51] [Discriminator Loss - -0.282296359539032] [Gradient Penalty - 0.23086202144622803] [Generator Loss - -0.2975289821624756
[Step - 101] [Discriminator Loss - -0.15392351150512695] [Gradient Penalty - 0.36093419790267944] [Generator Loss - -0.28858622908592224
[Step - 151] [Discriminator Loss - -0.2975296080112457] [Gradient Penalty - 0.23895010352134705] [Generator Loss - -0.3097977638244629
[Step - 201] [Discriminator Loss - -0.1654680073261261] [Gradient Penalty - 0.3108840882778168] [Generator Loss - -0.31040292978286743
[Step - 251] [Discriminator Loss - -0.2514588534832001] [Gradient Penalty - 0.22129932045936584] [Generator Loss - -0.3116726875305176
[Step - 301] [Discriminator Loss - -0.3041006922721863] [Gradient Penalty - 0.19552111625671387] [Generator Loss - -0.31511592864990234
[Step - 351] [Discriminator Loss - -0.2510405480861664] [Gradient Penalty - 0.20029541850090027] [Generator Loss - -0.303914338350296
[Step - 401] [Discriminator Loss - -0.26676756


Epoch 5
[Step - 51] [Discriminator Loss - -0.3914884626865387] [Gradient Penalty - 0.09548798203468323] [Generator Loss - -0.3184526264667511
[Step - 101] [Discriminator Loss - -0.24145734310150146] [Gradient Penalty - 0.2619640827178955] [Generator Loss - -0.31559497117996216
[Step - 151] [Discriminator Loss - -0.3916112184524536] [Gradient Penalty - 0.14079882204532623] [Generator Loss - -0.33160316944122314
[Step - 201] [Discriminator Loss - -0.3325386941432953] [Gradient Penalty - 0.22937622666358948] [Generator Loss - -0.27952852845191956
[Step - 251] [Discriminator Loss - -0.31392931938171387] [Gradient Penalty - 0.2324964851140976] [Generator Loss - -0.30232828855514526
[Step - 301] [Discriminator Loss - -0.38522884249687195] [Gradient Penalty - 0.13063767552375793] [Generator Loss - -0.32546061277389526
[Step - 351] [Discriminator Loss - -0.3081021308898926] [Gradient Penalty - 0.2522018551826477] [Generator Loss - -0.3172709345817566
[Step - 401] [Discriminator Loss - -0.3595

...


Epoch 65
[Step - 51] [Discriminator Loss - -0.1492500752210617] [Gradient Penalty - 0.10766007006168365] [Generator Loss - -0.43428313732147217
[Step - 101] [Discriminator Loss - -0.24923595786094666] [Gradient Penalty - 0.05247282236814499] [Generator Loss - -0.4369082450866699
[Step - 151] [Discriminator Loss - -0.20162278413772583] [Gradient Penalty - 0.04999465495347977] [Generator Loss - -0.4156107008457184
[Step - 201] [Discriminator Loss - -0.13416920602321625] [Gradient Penalty - 0.11979033052921295] [Generator Loss - -0.3794742226600647
[Step - 251] [Discriminator Loss - -0.22948898375034332] [Gradient Penalty - 0.058880969882011414] [Generator Loss - -0.3660285174846649
[Step - 301] [Discriminator Loss - -0.12137601524591446] [Gradient Penalty - 0.06821196526288986] [Generator Loss - -0.41418391466140747
[Step - 351] [Discriminator Loss - -0.11893750727176666] [Gradient Penalty - 0.1466047614812851] [Generator Loss - -0.46828874945640564
[Step - 401] [Discriminator Loss - -0


Epoch 66
[Step - 51] [Discriminator Loss - -0.1778489351272583] [Gradient Penalty - 0.0950954481959343] [Generator Loss - -0.4703865945339203
[Step - 101] [Discriminator Loss - -0.2201688587665558] [Gradient Penalty - 0.07271057367324829] [Generator Loss - -0.29934877157211304
[Step - 151] [Discriminator Loss - -0.24837946891784668] [Gradient Penalty - 0.06762475520372391] [Generator Loss - -0.3101266324520111
[Step - 201] [Discriminator Loss - -0.13530224561691284] [Gradient Penalty - 0.10257437080144882] [Generator Loss - -0.4348844885826111
[Step - 251] [Discriminator Loss - -0.22362247109413147] [Gradient Penalty - 0.06974074989557266] [Generator Loss - -0.3661026060581207
[Step - 301] [Discriminator Loss - -0.15496283769607544] [Gradient Penalty - 0.10545694828033447] [Generator Loss - -0.36079299449920654
[Step - 351] [Discriminator Loss - -0.28138625621795654] [Gradient Penalty - 0.05570930242538452] [Generator Loss - -0.380915105342865
[Step - 401] [Discriminator Loss - -0.209


Epoch 67
[Step - 51] [Discriminator Loss - -0.23315221071243286] [Gradient Penalty - 0.04556924104690552] [Generator Loss - -0.4401368498802185
[Step - 101] [Discriminator Loss - -0.17022652924060822] [Gradient Penalty - 0.09694753587245941] [Generator Loss - -0.4412687420845032
[Step - 151] [Discriminator Loss - -0.22092953324317932] [Gradient Penalty - 0.07058069854974747] [Generator Loss - -0.40317782759666443
[Step - 201] [Discriminator Loss - -0.2261059433221817] [Gradient Penalty - 0.04296092316508293] [Generator Loss - -0.3895507752895355
[Step - 251] [Discriminator Loss - -0.2197938710451126] [Gradient Penalty - 0.06659440696239471] [Generator Loss - -0.5483545064926147
[Step - 301] [Discriminator Loss - -0.13859371840953827] [Gradient Penalty - 0.12666301429271698] [Generator Loss - -0.37696129083633423
[Step - 351] [Discriminator Loss - -0.24130813777446747] [Gradient Penalty - 0.048132169991731644] [Generator Loss - -0.42067086696624756
[Step - 401] [Discriminator Loss - -0


Epoch 68
[Step - 51] [Discriminator Loss - -0.19393575191497803] [Gradient Penalty - 0.11401597410440445] [Generator Loss - -0.38117194175720215
[Step - 101] [Discriminator Loss - -0.22397871315479279] [Gradient Penalty - 0.0559794157743454] [Generator Loss - -0.3626275062561035
[Step - 151] [Discriminator Loss - -0.21815964579582214] [Gradient Penalty - 0.07134804874658585] [Generator Loss - -0.3626192808151245
[Step - 201] [Discriminator Loss - -0.17747452855110168] [Gradient Penalty - 0.060463763773441315] [Generator Loss - -0.44751644134521484
[Step - 251] [Discriminator Loss - -0.2192102074623108] [Gradient Penalty - 0.05987176299095154] [Generator Loss - -0.3363315463066101
[Step - 301] [Discriminator Loss - -0.11611461639404297] [Gradient Penalty - 0.12703275680541992] [Generator Loss - -0.42289507389068604
[Step - 351] [Discriminator Loss - -0.2580978572368622] [Gradient Penalty - 0.03718546777963638] [Generator Loss - -0.40998637676239014
[Step - 401] [Discriminator Loss - -0

KeyboardInterrupt: ignored